## Hello World using Flask

In [1]:
import os
hello_world_script_file = os.path.join(os.path.pardir,'src','models','hello_world_api.py')

In [2]:
%%writefile $hello_world_script_file

from flask import Flask,request
app = Flask(__name__)
@app.route('/api',methods=['POST'])
def say_hello():
    data = request.get_json(force=True)
    name = data['name']
    return 'hello {0}'.format(name)

if __name__ == '__main__':
    app.run(port=3000,debug=True)

Overwriting ../src/models/hello_world_api.py


In [3]:
import json
import requests

In [15]:
url = 'http://localhost:3000/api'
data = json.dumps({'name' : 'bhavana'}) #creating json object for sending data
r = requests.post(url,data) #response from url

In [16]:
r.text

'hello bhavana'

## Maching Learning with API

#### Building API

In [6]:
import os
machine_learning_api_script_file = os.path.join(os.path.pardir,'src','models','machine_learning_api.py')

In [7]:
%%writefile $machine_learning_api_script_file

from flask import Flask,request
import pandas as pd
import numpy as np
import json
import pickle
import os

app = Flask(__name__)

model_path = os.path.join(os.path.pardir,os.path.pardir,'models')
model_filepath = os.path.join(model_path,'lr_model.pkl')
scaler_filepath = os.path.join(model_path,'lr_scaler.pkl')

scaler = pickle.load(open(scaler_filepath,'rb'))
model = pickle.load(open(model_filepath,'rb'))

columns = ["Age", "Fare", "FamilySize",
       "IsMother", "IsMale", "Deck_A", "Deck_B", "Deck_C", "Deck_D", 
       "Deck_E", "Deck_F", "Deck_G", "Deck_z", "Pclass_1", "Pclass_2", 
       "Pclass_3", "Title_Lady", "Title_Master", "Title_Miss", "Title_Mr", 
       "Title_Mrs", "Title_Officer", "Title_Sir", "Fare_category_very_low", 
       "Fare_category_low", "Fare_category_high", "Fare_category_very_high", "Embarked_C", 
       "Embarked_Q", "Embarked_S", "AgeState_Adult", "AgeState_Child"]

@app.route('/api',methods=['POST'])
def make_prediction():
    # read json object and convert to json string
    data = json.dumps(request.get_json(force=True))
    # create dataframe using json string
    df = pd.read_json(data)
    # extract passengerIds 
    passenger_ids = df['PassengerId'].ravel()
    actuals = df['Survived'].ravel()
    # convert data to matrix form
    X = df[columns].as_matrix().astype('float')
    X_scaled = scaler.transform(X)
    predictions = model.predict(X_scaled)
    df_response = pd.DataFrame({'PassengerId': passenger_ids, 'Predictions':predictions, 'Actual':actuals})
    return df_response.to_json()

if __name__ == '__main__':
    app.run(port=3000, debug=True)

Overwriting ../src/models/machine_learning_api.py


#### Invoking API using requests

In [8]:
import os
import pandas as pd
processed_data_path = os.path.join(os.path.pardir,'data','processed')
train_file_path = os.path.join(processed_data_path,'train.csv')
train_df = pd.read_csv(train_file_path)


In [9]:
survived_passengers = train_df[train_df['Survived'] == 1][:5]

In [10]:
survived_passengers

,PassengerId,Survived,Age,Fare,FamilySize,IsMother,IsMale,Deck_A,Deck_B,Deck_C,...,Title_Sir,Fare_category_very_low,Fare_category_low,Fare_category_high,Fare_category_very_high,Embarked_C,Embarked_Q,Embarked_S,AgeState_Adult,AgeState_Child
1,2,1,38.0,71.2833,2,0,0,0,0,1,...,0,0,0,0,1,1,0,0,1,0
2,3,1,26.0,7.9250,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
3,4,1,35.0,53.1000,2,0,0,0,0,1,...,0,0,0,0,1,0,0,1,1,0
8,9,1,27.0,11.1333,3,1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
9,10,1,14.0,30.0708,2,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1


In [11]:
import requests
def make_api_request(data):
    url = 'http://localhost:3000/api'
    r = requests.post(url,data)
    return r.json()

In [12]:
make_api_request(survived_passengers.to_json())

{'Actual': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1},
 'PassengerId': {'0': 2, '1': 3, '2': 4, '3': 9, '4': 10},
 'Predictions': {'0': 1, '1': 1, '2': 1, '3': 1, '4': 1}}

In [13]:
result = make_api_request(train_df.to_json())
df_result = pd.read_json(json.dumps(result))
df_result.head()

,Actual,PassengerId,Predictions
0,0,1,0
1,1,2,1
10,1,108,0
100,0,190,0
101,1,20,1


In [14]:
import numpy as np
np.mean(df_result.Actual == df_result.Predictions)

0.8383838383838383